## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Brae,GB,2021-08-01 08:20:42,60.3964,-1.3530,51.71,78,35,18.50
1,1,Ushuaia,AR,2021-08-01 08:21:34,-54.8000,-68.3000,42.46,65,20,13.80
2,2,Butaritari,KI,2021-08-01 08:21:44,3.0707,172.7902,83.30,70,23,7.18
3,3,Charleston,US,2021-08-01 08:20:34,32.7766,-79.9309,84.22,93,75,7.00
4,4,Saskylakh,RU,2021-08-01 08:21:45,71.9167,114.0833,83.77,24,77,9.82


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Butaritari,KI,2021-08-01 08:21:44,3.0707,172.7902,83.30,70,23,7.18
3,3,Charleston,US,2021-08-01 08:20:34,32.7766,-79.9309,84.22,93,75,7.00
4,4,Saskylakh,RU,2021-08-01 08:21:45,71.9167,114.0833,83.77,24,77,9.82
6,6,Luleburgaz,TR,2021-08-01 08:21:46,41.4039,27.3592,88.97,33,0,5.32
11,11,Atuona,PF,2021-08-01 08:19:26,-9.8000,-139.0333,77.32,64,100,14.05
15,15,Puerto Ayora,EC,2021-08-01 08:17:56,-0.7393,-90.3518,70.56,90,6,7.00
16,16,East London,ZA,2021-08-01 08:21:49,-33.0153,27.9116,79.74,13,0,9.22
17,17,Rikitea,PF,2021-08-01 08:21:49,-23.1203,-134.9692,70.95,71,95,22.84
18,18,Guerrero Negro,MX,2021-08-01 08:19:51,27.9769,-114.0611,71.06,83,7,9.24
26,26,Hambantota,LK,2021-08-01 08:21:19,6.1241,81.1185,87.12,63,100,25.12


In [13]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

282


Unnamed: 0    282
City          282
Country       278
Date          282
Lat           282
Lng           282
Max Temp      282
Humidity      282
Cloudiness    282
Wind Speed    282
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

278


,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Butaritari,KI,2021-08-01 08:21:44,3.0707,172.7902,83.30,70,23,7.18
3,3,Charleston,US,2021-08-01 08:20:34,32.7766,-79.9309,84.22,93,75,7.00
4,4,Saskylakh,RU,2021-08-01 08:21:45,71.9167,114.0833,83.77,24,77,9.82
6,6,Luleburgaz,TR,2021-08-01 08:21:46,41.4039,27.3592,88.97,33,0,5.32
11,11,Atuona,PF,2021-08-01 08:19:26,-9.8000,-139.0333,77.32,64,100,14.05


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Butaritari,KI,83.30,3.0707,172.7902,
3,Charleston,US,84.22,32.7766,-79.9309,
4,Saskylakh,RU,83.77,71.9167,114.0833,
6,Luleburgaz,TR,88.97,41.4039,27.3592,
11,Atuona,PF,77.32,-9.8000,-139.0333,
15,Puerto Ayora,EC,70.56,-0.7393,-90.3518,
16,East London,ZA,79.74,-33.0153,27.9116,
17,Rikitea,PF,70.95,-23.1203,-134.9692,
18,Guerrero Negro,MX,71.06,27.9769,-114.0611,
26,Hambantota,LK,87.12,6.1241,81.1185,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Butaritari,KI,83.30,3.0707,172.7902,Isles Sunset Lodge
3,Charleston,US,84.22,32.7766,-79.9309,The Mills House Wyndham Grand Hotel
4,Saskylakh,RU,83.77,71.9167,114.0833,
6,Luleburgaz,TR,88.97,41.4039,27.3592,alparslan rant a car
11,Atuona,PF,77.32,-9.8000,-139.0333,Villa Enata
15,Puerto Ayora,EC,70.56,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,East London,ZA,79.74,-33.0153,27.9116,Tu Casa
17,Rikitea,PF,70.95,-23.1203,-134.9692,People ThankYou
18,Guerrero Negro,MX,71.06,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
26,Hambantota,LK,87.12,6.1241,81.1185,Bungalow 63


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))